<a href="https://colab.research.google.com/github/amina04/CNN-lstm/blob/main/CNN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **chargement des fichiers qui sont sur drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


la bibliothéque pandas nous donne le droit de travailler avec la format csv.
On utilise panda pour ouvrir la dataset

In [ ]:
import pandas as pd  

In [ ]:
import tensorflow as tf

**Donner les noms des column au dataset et spécifier la liste des  string contenu **

In [ ]:

featureV=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
          "num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
          "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
          "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", 
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
          "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]

flagV=['OTH','RSTOS0','SF','SH','RSTO','S2','S1','REJ','S3','RSTR','S0']

protocol_typeV=['tcp','udp','icmp']

serviceV=['http','smtp','finger','domain_u','auth','telnet','ftp','eco_i','ntp_u','ecr_i','other','private','pop_3','ftp_data',
                  'rje','time','mtp','link','remote_job','gopher','ssh','name','whois','domain','login','imap4','daytime','ctf','nntp',
                  'shell','IRC','nnsp','http_443','exec','printer','efs','courier','uucp','klogin','kshell','echo','discard','systat',
                  'supdup','iso_tsap','hostnames','csnet_ns','pop_2','sunrpc','uucp_path','netbios_ns','netbios_ssn','netbios_dgm',
                  'sql_net','vmnet','bgp','Z39_50','ldap','netstat','urh_i','X11','urp_i','pm_dump','tftp_u','tim_i','red_i','icmp',
                  'http_2784','harvest','aol','http_8001']

binary_attack=['normal','ipsweep', 'nmap', 'portsweep','satan', 'saint', 'mscan','back', 'land', 'neptune', 'pod', 'smurf',
               'teardrop', 'apache2', 'udpstorm', 'processtable','mailbomb','buffer_overflow', 'loadmodule', 'perl', 'rootkit',
               'xterm', 'ps', 'sqlattack','ftp_write', 'guess_passwd', 'imap', 'multihop','phf', 'spy', 'warezclient',
               'warezmaster','snmpgetattack','named', 'xlock', 'xsnoop','sendmail', 'httptunnel', 'worm', 'snmpguess']

multiclass_attack={ 'normal': 'normal',
        'probe': ['ipsweep.', 'nmap.', 'portsweep.','satan.', 'saint.', 'mscan.'],
        'dos': ['back.', 'land.', 'neptune.', 'pod.', 'smurf.','teardrop.', 'apache2.', 'udpstorm.', 'processtable.','mailbomb.'],
        'u2r': ['buffer_overflow.', 'loadmodule.', 'perl.', 'rootkit.','xterm.', 'ps.', 'sqlattack.'],
        'r2l': ['ftp_write.', 'guess_passwd.', 'imap.', 'multihop.','phf.', 'spy.', 'warezclient.', 'warezmaster.','snmpgetattack.',
                   'named.', 'xlock.', 'xsnoop.','sendmail.', 'httptunnel.', 'worm.', 'snmpguess.']}    

## **1.Load the data**

In [ ]:
data_train=pd.read_csv('/content/drive/MyDrive/data/nsl-kdd/KDDTrain+.txt',names=featureV)
data_valid=pd.read_csv('/content/drive/MyDrive/data/nsl-kdd/KDDTest+.txt',names=featureV)

In [ ]:
print(f"numbers of protocol_type: {len(data_train['protocol_type'].value_counts())}")
print(f"number of 'service' in train datasets: {len(data_train.service.value_counts())}")
print(f"number of 'flag' in train datasets: {len(data_train['flag'].value_counts())}")

numbers of protocol_type: 3
number of 'service' in train datasets: 70
number of 'flag' in train datasets: 11


In [ ]:
data_train.label.unique()

array(['normal', 'neptune', 'warezclient', 'ipsweep', 'portsweep',
       'teardrop', 'nmap', 'satan', 'smurf', 'pod', 'back',
       'guess_passwd', 'ftp_write', 'multihop', 'rootkit',
       'buffer_overflow', 'imap', 'warezmaster', 'phf', 'land',
       'loadmodule', 'spy', 'perl'], dtype=object)

In [ ]:
data_train.shape

(125973, 43)

In [ ]:
df_train_obj = data_train.iloc[:, :-2].select_dtypes(include='object')
df_train_num = data_train.iloc[:, :-2].select_dtypes(exclude='object')

print(f"shape of numeric features: {df_train_num.shape}")
print(f"shape of object features: {df_train_obj.shape}")

shape of numeric features: (125973, 38)
shape of object features: (125973, 3)


In [ ]:
df_valid_obj = data_valid.iloc[:, :-2].select_dtypes(include='object')
df_valid_num = data_valid.iloc[:, :-2].select_dtypes(exclude='object')

print(f"shape of numeric features: {df_valid_num.shape}")
print(f"shape of object features: {df_valid_obj.shape}")

shape of numeric features: (22544, 38)
shape of object features: (22544, 3)


In [ ]:
df_train_obj.head()

,protocol_type,service,flag
0,tcp,ftp_data,SF
1,udp,other,SF
2,tcp,private,S0
3,tcp,http,SF
4,tcp,http,SF


encoder les labels 0 pour normal et 1 attack

In [ ]:
data_train['label']=data_train['label'].replace(['normal.','normal'],0)
for i in range(len(binary_attack)):
  data_train['label'] = data_train['label'].replace(binary_attack[i], 1)
y_train=data_train['label']
data_valid['label']=data_valid['label'].replace(['normal.','normal'],0)
for i in range(len(binary_attack)):
  data_valid['label'] = data_valid['label'].replace(binary_attack[i], 1)
y_valid=data_valid['label']
#supprimer la colonne la bel de dataset car on a créer une copier déja ===>séparation
del data_train['label']
del data_valid['label']


In [ ]:
y_train.shape

(125973,)

## **2.Working with Validation data -Numericalization-**

In [ ]:
import numpy as np 
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

#encoder juste les les column qui contient string
data_train_enc = enc.fit_transform(df_train_obj).toarray()
train_enc_features = enc.get_feature_names_out(input_features=df_train_obj.columns)
data_valid_enc = enc.transform(df_valid_obj).toarray()
test_enc_features = enc.get_feature_names_out(input_features=df_valid_obj.columns)

#concatiner string avec num columns
X_train_enc = np.c_[df_train_num, data_train_enc]
X_valid_enc = np.c_[df_valid_num, data_valid_enc]
print(f"X_train_enc shape: {X_train_enc.shape}")
print(f"X_test_enc shape: {X_valid_enc.shape}")
######################################
#encoder les labels
y_train=pd.get_dummies(y_train)
y_valid=pd.get_dummies(y_valid)
y_train.head()

X_train_enc shape: (125973, 122)
X_test_enc shape: (22544, 122)


,0,1
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [ ]:
scaler = MinMaxScaler()
X_train_scaler = scaler.fit_transform(X_train_enc)
X_valid_scaler = scaler.fit_transform(X_valid_enc)
########################
y_train=scaler.fit_transform(y_train)
y_valid=scaler.fit_transform(y_valid)
y_train[0]

array([1., 0.])

In [ ]:
x_train = np.reshape(X_train_scaler, (X_train_scaler.shape[0], X_train_scaler.shape[1], 1))

x_train[0]

array([[0.00000000e+00],
       [3.55806412e-07],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [3.91389432e-03],
       [3.91389432e-03],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [1.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [5.88235294e-01],
       [9.80392157e-02],
       [1.70000000e-01],
       [3.00000000e-02],
       [1.70000000e-01],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [5.00000000e-02],
       [0.00000000e+00],
       [0.00000000e+00],
       [1.00000000e+00],


In [ ]:
y_valid.shape

(22544, 2)

In [ ]:
x_valid = np.reshape(X_valid_scaler, (X_valid_scaler.shape[0],X_valid_scaler.shape[1], 1))

y_valid.shape

(22544, 2)

## **3.Define the model**

In [ ]:
#instancier le modéle
from tensorflow.keras.models import Sequential
model=Sequential()
#Créer la couche entrée totalement connecté avec la couche dense
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, AveragePooling1D,BatchNormalization,SpatialDropout1D
from tensorflow.keras.layers import LeakyReLU

In [ ]:
model.add(Conv1D(32,kernel_size =5,padding="same",input_shape = (x_train.shape[1],1)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(AveragePooling1D(pool_size=2))
#model.add(SpatialDropout1D(0.05))
model.add(Dropout(0.05))
model.add(Flatten())
#model.add(Dropout(0.5))
#(83*32)/4=664
model.add(Dense(units=500))

model.add(LeakyReLU(alpha=0.1))
model.add(Dense(2, activation="softmax"))
model.summary() #sert afficher résumé

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 122, 32)           192       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 122, 32)           0         
                                                                 
 batch_normalization (BatchN  (None, 122, 32)          128       
 ormalization)                                                   
                                                                 
 average_pooling1d (AverageP  (None, 61, 32)           0         
 ooling1D)                                                       
                                                                 
 dropout (Dropout)           (None, 61, 32)            0         
                                                                 
 flatten (Flatten)           (None, 1952)              0

## **4.Compile the model**


In [ ]:
from tensorflow import keras
#opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy',keras.metrics.Precision()])

## **5.Fit the model**

In [ ]:
#model.fit(x_train, y_train, epochs = 100, batch_size =1000)
model.fit(x_train, y_train, epochs =80,batch_size =128,validation_data=(x_valid, y_valid))

Epoch 1/80
985/985 [==============================] - 51s 50ms/step - loss: 0.0483 - accuracy: 0.9832 - precision: 0.9832 - val_loss: 1.0815 - val_accuracy: 0.7930 - val_precision: 0.7930
Epoch 2/80
985/985 [==============================] - 41s 42ms/step - loss: 0.0216 - accuracy: 0.9925 - precision: 0.9925 - val_loss: 1.1534 - val_accuracy: 0.7859 - val_precision: 0.7859
Epoch 3/80
985/985 [==============================] - 43s 44ms/step - loss: 0.0182 - accuracy: 0.9937 - precision: 0.9937 - val_loss: 1.0253 - val_accuracy: 0.8050 - val_precision: 0.8050
Epoch 4/80
985/985 [==============================] - 42s 42ms/step - loss: 0.0159 - accuracy: 0.9944 - precision: 0.9944 - val_loss: 1.3749 - val_accuracy: 0.7773 - val_precision: 0.7773
Epoch 5/80
985/985 [==============================] - 42s 42ms/step - loss: 0.0151 - accuracy: 0.9948 - precision: 0.9948 - val_loss: 1.3504 - val_accuracy: 0.7748 - val_precision: 0.7748
Epoch 6/80
985/985 [==============================] - 41s 42